In [1]:
from pyqubo import Array, Binary, Model
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import requests
import json
import re
import time
import numpy as np
import scipy as sp

from lib.util import *

np.set_printoptions(precision=10, suppress=True)

In [2]:
def decode(configuration, dimension, precision, start_point):
    binary_code = create_binary_representated_vector(precision, start_point)
    nrows, ncols = dimension
    
    solution_dict = {}
    for key, value in configuration.items():
        if value:
            solution_dict[model.variables[int(key)]] = 1
        else:
            solution_dict[model.variables[int(key)]] = 0
            
    # binary_representation_columns is an array whose element are represented as a matrix
    # the matrix multiply binary_code would become a column
    binary_representation_columns = []
    for i in range(ncols):
        binary_representation_columns.append(np.zeros((nrows, precision)))

    for key, value in solution_dict.items():
        # get the name of column
        indexes = re.findall(r'(\d+)', key)
        indexes = [int(i) for i in indexes]
        col = indexes[0]
        binary_representation_columns[col][indexes[1]][indexes[2]] = value

    columns = []
    for mat in binary_representation_columns:
        columns.append(np.array((mat * np.matrix(binary_code).transpose()).transpose())[0])

    return np.matrix(np.column_stack(columns)), binary_representation_columns, binary_code

Step 1. Generate a positive definite matrix X

In [3]:
def generateX(dimension):
    symmetric_matrix = np.matrix(np.random.uniform(3, 5, size=(N, N)))
    X = (symmetric_matrix * symmetric_matrix.transpose())  / symmetric_matrix.trace()
    
    print(symmetric_matrix)
    
    return X

def generateRandomMatrix(dimension):
    return np.matrix(np.random.rand(dimension[0], dimension[1]) + 1) 

In [4]:
def get_solver_config(matrix_terms, sec):
    DA_Solver = {}
    DA_Solver["time_limit_sec"] = sec
    DA_Solver["penalty_coef"] = 10000
    DA_Solver['gs_level'] = 50
    DA_Solver['gs_cutoff'] = 80000
    
    problem_body = {}
    problem_body["fujitsuDA3"] = DA_Solver
    problem_body["binary_polynomial"] = {'terms' : matrix_terms}
    
    return problem_body

In [30]:
def setupInitial(bit_columns, variables):
    guidance = {}
    for i in range(len(bit_columns)):
        nrows, ncols = bit_columns[i].shape
        
        for j in range(nrows):
            for k in range(ncols):
                name = 'col%d[%d][%d]' % (i, j, k)
                index = variables.index(name)
                if index == 1:
                    guidance[str(index)] = True
                else:
                    guidance[str(index)] = False
    return guidance

### Iteratively solve X-WG

In [21]:
N = 3
K = 2
precision = 24
starting_power = 0
w_matrix_dimension = (N, K)
g_matrix_dimension = (K, N)

Wp = generate_random_matrix(w_matrix_dimension)
Gp = generate_random_matrix(g_matrix_dimension)
X = Wp*Gp
print(X)

# check if X is positive definite
np.all(np.linalg.eigvals(X) > 0)

[[3.8602975417 4.195701378  4.8530044139]
 [4.2431516655 5.0238152419 5.4214862109]
 [3.8950382146 3.9912538236 4.8454306872]]


True

\begin{equation} \label{eq1}
\begin{split}
\lVert X-WG\rVert_{F}^2 & = Tr(X^TX) - Tr(X^TWG) - Tr(G^TW^TX) + Tr(G^TW^TWG) \\
 & = Tr(X^TX) - Tr(X^TWG) - Tr(W^TXG^T) + Tr(W^TWGG^T) \\
 & = Tr(X^TX) - Tr(GX^TW) - Tr(W^TXG^T) + Tr(W^TWGG^T)
\end{split}
\end{equation}

In [35]:
G = generate_random_matrix(g_matrix_dimension)
iterations = 0
max_iterations = 30
initial_columns = None
w_columns = g_columns = None
while iterations < max_iterations:
    if iterations % 2 == 0:
        W = generate_target_matrix(w_matrix_dimension, precision, starting_power)
#         hamiltonian = np.trace(W.transpose()*W*G*G.transpose()) - np.trace(X.transpose()*W*G) - np.trace(W.transpose()*X*G.transpose())
        initial_val = w_columns
    else:
        G = generate_target_matrix(g_matrix_dimension, precision, starting_power)
#         hamiltonian = np.trace(W.transpose()*W*G*G.transpose()) - np.trace(G*X.transpose()*W) - np.trace(W.transpose()*X*G.transpose())
        initial_val = g_columns
    
    hamiltonian = np.trace(np.inner(X-W*G, X-W*G))
    model = hamiltonian.compile()
    qubo, offset = model.to_qubo()
    matrix_term = get_matrix_term(qubo, model.variables)
    print("Number of variables : ", len(model.variables))
    
    time_limit = 3
    config = get_solver_config(matrix_term, time_limit)
    
    # setup the initial config
    if initial_val:
        guidance = setupInitial(initial_val, model.variables)
        config['fujitsuDA3']['guidance_config'] = guidance
    
    job_id = post_solve(config).json()['job_id']
    time.sleep(time_limit)
    
    time.sleep(time_limit)
    solution = get_solution(job_id)
    while not 'qubo_solution' in solution:
        time.sleep(1)
        solution = get_solution(job_id)
        
    configuration = solution['qubo_solution']['solutions'][0]['configuration']
    progress = solution['qubo_solution']['progress']
    for i in range(len(progress)):
        print(f"{progress[i]['time']:.5f} : {progress[i]['energy']:.10f}")
        
    if iterations % 2 == 0:
        W, w_columns, w_binary_code = decode(configuration, w_matrix_dimension, precision, starting_power)
        
    else:
        G, g_columns, g_binary_code = decode(configuration, g_matrix_dimension, precision, starting_power)
        
    frobenius_norm = np.trace(np.inner(X-W*G, X-W*G))
    delete_job(job_id)
    
    print(f"{iterations} : {frobenius_norm}")
    
    iterations += 1

Number of variables :  144
{'message': 'success'}
0.13000 : -183.2219374747
0.13000 : -183.2219590175
0.23500 : -183.2219670212
0.23500 : -183.2219677613
0.23500 : -183.2219677613
0.34800 : -183.2219686219
0.34800 : -183.2219686448
0.34800 : -183.2219686779
0.56700 : -183.2219686779
0.56700 : -183.2219686782
0.68400 : -183.2219686782
0 : 0.05068702565659591
Number of variables :  144
{'message': 'success'}
0.13300 : -183.2611945439
0.13300 : -183.2622073556
0.24600 : -183.2625778063
0.24600 : -183.2625879969
0.35900 : -183.2626007550
0.35900 : -183.2626103303
0.46700 : -183.2626104644
0.46700 : -183.2626105683
1.36600 : -183.2626687640
1.48100 : -183.2626708182
1.48200 : -183.2626708202
1.48200 : -183.2626708208
2.39900 : -183.2627372122
2.51400 : -183.2628195705
2.51400 : -183.2628299022
2.62900 : -183.2628528085
2.74300 : -183.2628532860
2.74300 : -183.2628533736
2.85600 : -183.2628533744
2.85700 : -183.2628533761
2.85700 : -183.2628533761
2.97200 : -183.2628533764
1 : 0.009802327462

12 : 8.408995990296443e-07
Number of variables :  144
{'message': 'success'}
0.12600 : -183.2312798656
0.23600 : -183.2724109453
0.23700 : -183.2724715045
0.35400 : -183.2725940125
0.35400 : -183.2726078616
0.35400 : -183.2726152834
0.35400 : -183.2726153148
0.46500 : -183.2726479406
0.57800 : -183.2726535834
0.69000 : -183.2726552046
0.80200 : -183.2726553393
0.91700 : -183.2726553485
1.37200 : -183.2726553486
1.37200 : -183.2726553486
1.37200 : -183.2726553486
1.49000 : -183.2726553486
13 : 3.552434768353192e-07
Number of variables :  144
{'message': 'success'}
0.11100 : -183.2697580115
0.11100 : -183.2698010494
0.20700 : -183.2721515909
0.20700 : -183.2724208979
0.30600 : -183.2724804524
0.30700 : -183.2724804524
0.30700 : -183.2724804960
0.88700 : -183.2725253892
0.98400 : -183.2725262421
1.08100 : -183.2725262421
1.08200 : -183.2725262421
1.18300 : -183.2726482386
1.28100 : -183.2726555182
1.28100 : -183.2726555535
1.38000 : -183.2726555536
1.38000 : -183.2726555536
1.38100 : -183

24 : 2.952698411614567e-11
Number of variables :  144
{'message': 'success'}
0.11900 : -183.1755076779
0.11900 : -183.2719130043
0.22300 : -183.2726141770
0.22300 : -183.2726149981
0.33100 : -183.2726509579
0.33100 : -183.2726525627
0.43700 : -183.2726556360
0.54400 : -183.2726557023
0.54400 : -183.2726557026
0.54500 : -183.2726557036
0.65100 : -183.2726557037
1.18500 : -183.2726557037
1.29400 : -183.2726557037
1.29400 : -183.2726557037
2.95900 : -183.2726557038
3.00900 : -183.2726557038
3.00900 : -183.2726557038
4.32200 : -183.2726557038
25 : 1.3019353485694065e-11
Number of variables :  144
{'message': 'success'}
0.13000 : -183.2701107076
0.13000 : -183.2708786961
0.13000 : -183.2715274795
0.24700 : -183.2722567092
0.24700 : -183.2724592181
0.36600 : -183.2724831788
0.36600 : -183.2724831788
0.48600 : -183.2724832219
0.48600 : -183.2724832219
0.84600 : -183.2726527063
0.84600 : -183.2726552727
0.84600 : -183.2726556270
0.96000 : -183.2726556381
0.96100 : -183.2726556381
1.44700 : -18

In [32]:
config

{'fujitsuDA3': {'time_limit_sec': 3,
  'penalty_coef': 10000,
  'gs_level': 50,
  'gs_cutoff': 80000},
 'binary_polynomial': {'terms': [{'c': 7.621815378743416e-07, 'p': [0, 95]},
   {'c': 0.0007804738947833258, 'p': [21, 83]},
   {'c': 1.9965293848133847e-07, 'p': [5, 15]},
   {'c': -0.5649782862044949, 'p': [17, 17]},
   {'c': 1.4537459142195542e-12, 'p': [49, 122]},
   {'c': 0.013084454976312998, 'p': [38, 47]},
   {'c': 9.303973851005147e-11, 'p': [48, 129]},
   {'c': 1.9965293848133847e-07, 'p': [52, 64]},
   {'c': 1.072850787656738e-10, 'p': [96, 105]},
   {'c': -4.756319556565996e-06, 'p': [72, 72]},
   {'c': 4.8779618423957865e-05, 'p': [67, 129]},
   {'c': 0.00019511847369583146, 'p': [31, 118]},
   {'c': 1.8607947702010294e-10, 'p': [53, 125]},
   {'c': 4.291403150626952e-10, 'p': [77, 78]},
   {'c': 8.582806301253904e-10, 'p': [77, 79]},
   {'c': 8.582806301253904e-10, 'p': [74, 82]},
   {'c': 4.8779618423957865e-05, 'p': [55, 141]},
   {'c': 2.9074918284391084e-12, 'p': [49

In [38]:
Wp

matrix([[1.5662531651, 1.8651411978],
        [1.7428174776, 1.6772655989],
        [1.237025753 , 1.5456730135]])

In [37]:
G

matrix([[1.7884216309, 1.3509120941, 1.6908321381],
        [1.4725818634, 1.2377662659, 1.2358760834]])

In [39]:
Gp

matrix([[1.7969143471, 1.6094669882, 1.3845530321],
        [1.9477317971, 1.3745147342, 1.9620380331]])

In [41]:
print(X)
print(Wp*Gp - X)

[[6.4472176006 5.0844968222 5.828038536 ]
 [6.3985572691 5.1104334756 5.7038821198]
 [5.2333857996 4.1155024444 4.7453969964]]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [11]:
def getOffset(matrix_columns, binary_code, power):
    columns = []
    for i in range(len(matrix_columns)):
        columns.append(np.matrix(matrix_columns[i][:, :power]) * np.matrix(binary_code[:power]).transpose())
    
    matrix = np.matrix(np.column_stack(columns))
    return matrix

In [ ]:
precision = 8
starting_power = 0
iteration_inner = 0
W_offset = np.matrix(np.zeros(w_matrix_dimension))
G_offset = np.matrix(np.zeros(g_matrix_dimension))

X = optimal_w*optimal_g
G = generateRandomMatrix(g_matrix_dimension)
while iteration_inner < 10:
    if iteration_inner % 2 == 0:
        W = generateTargetMatrix(w_matrix_dimension, precision, starting_power)
    else:
        G = generateTargetMatrix(g_matrix_dimension, precision, starting_power)
    
    hamiltonian = np.trace(np.inner(X-(W_offset+W)*(G_offset+G), X-(W_offset+W)*(G_offset+G)))
    model = hamiltonian.compile()
    qubo, offset = model.to_qubo()
    matrix_term = get_matrix_term(qubo, model.variables)
    
    time_limit = 1
    config = solverConfig(matrix_term, time_limit)
    job_id = postSolve(config).json()['job_id']
    
    time.sleep(time_limit)
    solution = getSolution(job_id)
    while not 'qubo_solution' in solution:
        time.sleep(1)
        solution = getSolution(job_id)
        
    configuration = solution['qubo_solution']['solutions'][0]['configuration']
    progress = solution['qubo_solution']['progress']
    
    for i in range(len(progress)):
        print(f"{progress[i]['time']:.5f} : {progress[i]['energy']:.10f}")
        
    if iteration_inner % 2 == 0:
        W, w_columns, w_binary_code = decode(configuration, w_matrix_dimension, precision, starting_power)
    else:
        G, g_columns, g_binary_code = decode(configuration, g_matrix_dimension, precision, starting_power)


    frobenius_norm = np.trace(np.inner(X-(W_offset+W)*(G_offset+G), X-(W_offset+W)*(G_offset+G)))
    deleteJob(job_id)
    print(f"{iteration_inner} : {frobenius_norm:.10f}")
    
    iteration_inner += 1
    
print(W + W_offset)
W_offset += getOffset(w_columns, w_binary_code, 7)
G_offset += getOffset(g_columns, g_binary_code, 7)
print(W_offset)
starting_power -= 7
iteration_outer += 1

In [ ]:
col0 = np.matrix(w_columns[0])*np.matrix(w_binary_code).transpose()
col1 = np.matrix(w_columns[1])*np.matrix(w_binary_code).transpose()
print(col0)
print(col1)

In [ ]:
W_offset = getOffset(w_columns, w_binary_code, 6)

In [ ]:
W_target = generateTargetMatrix(w_matrix_dimension, 2, -6)

In [20]:
hamiltonian = np.trace(np.inner((W - (W_offset + W_target)), (W - (W_offset + W_target))))
model = hamiltonian.compile()
qubo, offset = model.to_qubo()
matrix_term = get_matrix_term(qubo, model.variables)

time_limit = 1
config = solverConfig(matrix_term, time_limit)
job_id = postSolve(config).json()['job_id']

time.sleep(time_limit)
solution = getSolution(job_id)
while not 'qubo_solution' in solution:
    time.sleep(2)
    solution = getSolution(job_id)

configuration = solution['qubo_solution']['solutions'][0]['configuration']
progress = solution['qubo_solution']['progress']


for i in range(len(progress)):
    print(f"{progress[i]['time']:.5f} : {progress[i]['energy']:.10f}")

W_target, w_columns, w_binary_code = decode(configuration, w_matrix_dimension, 2, -6)


frobenius_norm = np.trace(np.inner((W - (W_offset + W_target)), (W - (W_offset + W_target))))
deleteJob(job_id)
print(f"{frobenius_norm:.10f}")

NameError: name 'W_target' is not defined

In [21]:
optimal_w = generateRandomMatrix(w_matrix_dimension)
optimal_g = generateRandomMatrix(g_matrix_dimension)
X = optimal_w * optimal_g
print(optimal_w)
print(optimal_g)
print(X)
np.all(np.linalg.eigvals(X) > 0)

[[1.5719619848 1.4553564905]
 [1.7545381762 1.7644208029]
 [1.3158479894 1.8455678389]]
[[1.6542723988 1.0515121572 1.4842742254]
 [1.1753721271 1.7960072972 1.6380869454]]
[[4.3110387774 4.2667680146 4.7172231253]
 [4.9763351098 5.0138308599 5.4944904758]
 [4.3460000064 4.698283464  4.9762798389]]


True

In [23]:
# preliminary for solving
problem_body = {}
DA_Solver = {}
DA_Solver["time_limit_sec"]= 5
DA_Solver["num_run"] = 16
DA_Solver["num_group"] = 16
DA_Solver['gs_level'] = 50
DA_Solver['gs_cutoff'] = 80000
problem_body["fujitsuDA3"]=DA_Solver

Ws = []
Gs = []
G = generateRandomMatrix(g_matrix_dimension)
# X = generateX(x_matrix_dimension)
# X = optimal_w * optimal_g
print(X)


precision = 20
iteration_outer = 0
starting_power = 0
W_offset = np.matrix(np.zeros(w_matrix_dimension))
G_offset = np.matrix(np.zeros(g_matrix_dimension))

offset_bit = 13

inner_iterations = [
    {
        'time' : 6,
        'iteration' : 15
    },
    {
        'time' : 3,
        'iteration' : 6
    },
    {
        'time' : 2,
        'iteration' : 6
    },
    {
        'time' : 2,
        'iteration' : 6
    }
]

W_initial_vals = {}
G_initial_vals = {}

while iteration_outer < 2:
    
    iteration_inner = 0
    while iteration_inner < inner_iterations[iteration_outer]['iteration']:
        if iteration_inner % 2 == 0:
            W = generateTargetMatrix(w_matrix_dimension, precision, starting_power)
        else:
            G = generateTargetMatrix(g_matrix_dimension, precision, starting_power)
            
        hamiltonian = np.trace(np.inner(X-(W_offset+W)*(G_offset+G), X-(W_offset+W)*(G_offset+G)))
        
        
#         hamiltonian = np.trace(np.inner(X-W*G, X-W*G))

        model = hamiltonian.compile()
        qubo, offset = model.to_qubo()
        matrix_term = get_matrix_term(qubo, model.variables)
        
        
        
            
            
        
        print(f"number of variables : {len(model.variables)}")
        
        time_limit = inner_iterations[iteration_outer]['time']
        config = solverConfig(matrix_term, time_limit)
        
        if iteration_inner % 2 == 0 and len(W_initial_vals):
            initial_vals = W_initial_vals
        elif iteration_inner % 2 == 1 and len(G_initial_vals):
            initial_vals = G_initial_vals
        else:
            initial_vals = None
        
        if initial_vals:
            guidance = {}
            for key, value in initial_vals.items():
                for row in range(value.shape[0]):
                    for col in range(value.shape[1]):
                        variable_name = f'{key}[{row}][{col}]'
                        index = model.variables.index(variable_name)
                        if value[row][col] == 1:
                            guidance[index] = True
                        else:
                            guidance[index] = False
            config['fujitsuDA3']['fixed_config'] = guidance
        
        job_id = postSolve(config).json()['job_id']
        
        time.sleep(time_limit)
        solution = getSolution(job_id)
        while not 'qubo_solution' in solution:
            time.sleep(2)
            solution = getSolution(job_id)
        
        configuration = solution['qubo_solution']['solutions'][0]['configuration']
        progress = solution['qubo_solution']['progress']
        

        for i in range(len(progress)):
            print(f"{progress[i]['time']:.5f} : {progress[i]['energy']:.10f}")
        
        if iteration_inner % 2 == 0:
            W, w_columns, w_binary_code = decode(configuration, w_matrix_dimension, precision, starting_power)
        else:
            G, g_columns, g_binary_code = decode(configuration, g_matrix_dimension, precision, starting_power)
            
            
        frobenius_norm = np.trace(np.inner(X-(W_offset+W)*(G_offset+G), X-(W_offset+W)*(G_offset+G)))
#         frobenius_norm = np.trace(np.inner(X-W*G, X-W*G))
        deleteJob(job_id)
        print(f"{iteration_outer}-{iteration_inner} : {frobenius_norm}")
        
        
        iteration_inner += 1
     

    
    print(W + W_offset)
    Ws.append(W+W_offset)
    W_offset += getOffset(w_columns, w_binary_code, precision - 2)
    G_offset += getOffset(g_columns, g_binary_code, precision - 2)
    
    
    print(W_offset)
    starting_power -= offset_bit
    iteration_outer += 1
    precision = 2
    
    G = G - G_offset
    
    
    
    print("W : ")
    for col_indx in range(len(w_columns)):
        W_initial_vals[f'col{col_indx}'] = w_columns[col_indx][:, -2:]
        print(w_columns[col_indx][:, -2:])
    
    print("===================")
    print("G : ")
    for col_indx in range(len(g_columns)):
        G_initial_vals[f'col{col_indx}'] = g_columns[col_indx][:, -2:]
        print(g_columns[col_indx][:, -2:])
        
#     print(w_columns)
#     print(g_columns)
#     offset_bit = 6

[[4.3110387774 4.2667680146 4.7172231253]
 [4.9763351098 5.0138308599 5.4944904758]
 [4.3460000064 4.698283464  4.9762798389]]
number of variables : 120
{'message': 'success'}
0.11500 : -201.8220879561
0.11500 : -201.8220913215
0.11500 : -201.8220913215
0-0 : 3.0372485267533347
number of variables : 120
{'message': 'success'}
0.11100 : -204.8049123281
0.20500 : -204.8130125965
0.20500 : -204.8130139237
0.20500 : -204.8142859188
0.20500 : -204.8142949137
0.29500 : -204.8145710887
0.29500 : -204.8145998790
0.29500 : -204.8146085848
0.39200 : -204.8146264637
0.39200 : -204.8146386969
0.39200 : -204.8146459124
0.48400 : -204.8146503719
0.48400 : -204.8146529710
0.48400 : -204.8146538544
0.57600 : -204.8146544606
0.57600 : -204.8146604053
0.66900 : -204.8146604053
0.66900 : -204.8146607184
0.76000 : -204.8146638361
0.76000 : -204.8146656090
0.76000 : -204.8146657303
0.85100 : -204.8146657303
0.94300 : -204.8146658067
1.03300 : -204.8146681893
1.03300 : -204.8146682597
1.03300 : -204.8146683

0-13 : 3.2892532707504e-05
number of variables : 120
{'message': 'success'}
0.12700 : -204.8588099686
0.12800 : -204.8590532416
0.23600 : -204.8592774547
0.23600 : -204.8592867826
0.23600 : -204.8593147837
0.35000 : -204.8593205603
0.35000 : -204.8593205615
0.35000 : -204.8593211217
0.46000 : -204.8593212214
0.46000 : -204.8593212214
0.46000 : -204.8593212267
0.46000 : -204.8593212267
0.56900 : -204.8593212267
0-14 : 1.862156106029529e-05
[[1.4291172028 1.8159732819]
 [1.7820892334 1.9999980927]
 [1.9999980927 1.4337844849]]
[[1.4291152954 1.8159713745]
 [1.7820892334 1.9999923706]
 [1.9999923706 1.4337844849]]
W : 
[[0. 1.]
 [0. 0.]
 [1. 1.]]
[[0. 1.]
 [1. 1.]
 [0. 0.]]
G : 
[[1. 1.]
 [1. 0.]]
[[0. 0.]
 [1. 0.]]
[[1. 0.]
 [1. 0.]]
number of variables : 12
{'message': 'success'}
0.07200 : -0.0000006634
1-0 : 1.7992909468994123e-05
number of variables : 12
{'message': 'success'}
1-1 : 1.802052690044959e-05
number of variables : 12
{'message': 'success'}
0.07000 : -0.0000007124
1-2 : 1.8

matrix([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [ ]:
Ws[0]

In [ ]:
Ws[0] - Ws[1]

In [ ]:
print(binaryRepresentation(0.8046875, 8))
print(binaryRepresentation(0.796875, 8))
# print(binaryRepresentation(1.99890137, 8))

In [ ]:
w_columns[0][:, -4:]

In [ ]:
W

In [ ]:
for i in range(len(Gs)):
    print(Gs[i])

In [ ]:
print(W)
print(optimal_w)
print(np.trace(np.inner(W-optimal_w, W-optimal_w)))

In [ ]:
print(G)
print(optimal_g)
print(np.trace(np.inner(G-optimal_g, G-optimal_g)))

In [ ]:
print(W*G)
print(X)
print(X-W*G)
print(np.inner(X-W*G, X-W*G))

In [ ]:
nrows, ncols = optimal_g.shape
columns = {}
for col_i in range(ncols):
    col = np.array(optimal_g[:, col_i].transpose())[0]
    print(col)
    
    binary_representation_column = []
    for r in col:
        integer, floating= binary_representation(r, precision - 1)
        binary_representation_column.append(integer + floating)
    columns[f'col{col_i}'] = np.matrix(binary_representation_column)



In [ ]:
def representMatrixInTermsBinary(matrix, precision):
    nrows, ncols = matrix.shape
    columns = {}
    for col_i in range(ncols):
        col = np.array(matrix[:, col_i].transpose())[0]
        
        binary_representation_column = []
        for r in col:
            integer, floating = binary_representation(r, precision - 1)
            binary_representation_column.append(integer + floating)
        columns[f'col{col_i}'] = np.array(binary_representation_column)
#         print(columns[f'col{col_i}'].shape)
    return columns

In [ ]:
N = 3
K = 2
precision = 20
starting_power = 0
w_matrix_dimension = (N, K)
g_matrix_dimension = (K, N)
x_matrix_dimension = (N, N)
optimal_w = generateRandomMatrix(w_matrix_dimension)
optimal_g = generateRandomMatrix(g_matrix_dimension)
X = optimal_w * optimal_g
print(optimal_w)
print(optimal_w)
print(X)

In [ ]:
W = generateTargetMatrix(w_matrix_dimension, precision, starting_power)
G = generateRandomMatrix(g_matrix_dimension)

hamiltonian = np.trace(np.inner(X - W*G, X-W*G))

# compile
start_time = time.time()

model = hamiltonian.compile()
qubo, offset = model.to_qubo()
matrix_term = get_matrix_term(qubo, model.variables)

end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time, " seconds")
print("Number of variables: ", len(model.variables))

In [ ]:
# solve
problem_body = {}
DA_Solver = {}
DA_Solver["time_limit_sec"]=10
DA_Solver["penalty_coef"]=10000
# DA_Solver["num_run"] = 16
# DA_Solver["num_group"] = 16
DA_Solver['gs_level'] = 50
DA_Solver['gs_cutoff'] = 80000
problem_body["fujitsuDA3"]=DA_Solver
problem_body["binary_polynomial"] = {'terms' : matrix_term}
response = postSolve(problem_body)
job_id = response.json()['job_id']

# get solution
time.sleep(problem_body["fujitsuDA3"]["time_limit_sec"])
solution = getSolution(job_id)
while not 'qubo_solution' in solution:
    time.sleep(2)
    solution = getSolution(job_id)

In [ ]:
configuration = solution['qubo_solution']['solutions'][0]['configuration']
progress = solution['qubo_solution']['progress']
# print(progress)

W = decode(configuration, w_matrix_dimension, precision, starting_power)
print(np.trace(np.inner(X-W*G, X-W*G)))

deleteJob(job_id)

In [ ]:
G_target = generateTargetMatrix(g_matrix_dimension, precision, starting_power)
hamiltonian = np.trace(np.inner(X - W*G_target, X-W*G_target))

# compile
start_time = time.time()

model = hamiltonian.compile()
qubo, offset = model.to_qubo()
matrix_term = get_matrix_term(qubo, model.variables)

end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time, " seconds")
print("Number of variables: ", len(model.variables))

In [ ]:
columns = representMatrixInTermsBinary(G, precision)
guidance = {}
for i in range(len(model.variables)):
    var = model.variables[i]
    key = re.findall(r'(col\d+)', var)[0]
    
    indexes = re.findall(r'\[(\d+)\]', var)
    indexes = [int(i) for i in indexes]
    
    if columns[key][indexes[0]][indexes[1]] == 1:
        guidance[str(i)] = True
    else:
        guidance[str(i)] = False